In [1]:
# Imports

import pandas as pd  
import numpy as np    
import re
import lxml
import requests

In [2]:
cardapio_beleaf = requests.get("https://www.beleaf.com.br/cardapio-comida-congelada")

cardapio_beleaf

<Response [200]>

In [3]:
cardapio_beleaf.content

b'<!DOCTYPE html >\r\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:vtex="http://www.vtex.com.br/2009/vtex-common" xmlns:vtex.cmc="http://www.vtex.com.br/2009/vtex-commerce" lang="pt-br">\r\n  <head><meta name="language" content="pt-BR" />\n<meta name="country" content="BRA" />\n<meta name="currency" content="R$" />\n<style>.async-hide { opacity: 0 !important} </style><title>Cardapio-comida-congelada \xe2\x80\x93 veganja</title><script type="text/javascript" language="javascript">var jscheckoutUrl = \'https://www.beleaf.com.br/checkout/#/cart\';var jscheckoutAddUrl = \'https://www.beleaf.com.br/checkout/cart/add\';var jscheckoutGiftListId = \'\';var jsnomeSite = \'veganja\';var jsnomeLoja = \'veganja\';var jssalesChannel = \'1\';var defaultStoreCurrency = \'R$\';var localeInfo = {"CountryCode":"BRA","CultureCode":"pt-BR","CurrencyLocale":{"RegionDisplayName":"Brazil","RegionName":"BR","RegionNativeName":"Brasil","TwoLetterIsoRegionName":"BR","CurrencyEnglishName":"Real","CurrencyNat

In [4]:
from bs4 import BeautifulSoup

beleaf_soup1 = BeautifulSoup(cardapio_beleaf.content, 'html.parser')

In [5]:
print(beleaf_soup1.prettify())

<!DOCTYPE html >
<html lang="pt-br" xmlns="http://www.w3.org/1999/xhtml" xmlns:vtex="http://www.vtex.com.br/2009/vtex-common" xmlns:vtex.cmc="http://www.vtex.com.br/2009/vtex-commerce">
 <head>
  <meta content="pt-BR" name="language"/>
  <meta content="BRA" name="country"/>
  <meta content="R$" name="currency"/>
  <style>
   .async-hide { opacity: 0 !important}
  </style>
  <title>
   Cardapio-comida-congelada – veganja
  </title>
  <script language="javascript" type="text/javascript">
   var jscheckoutUrl = 'https://www.beleaf.com.br/checkout/#/cart';var jscheckoutAddUrl = 'https://www.beleaf.com.br/checkout/cart/add';var jscheckoutGiftListId = '';var jsnomeSite = 'veganja';var jsnomeLoja = 'veganja';var jssalesChannel = '1';var defaultStoreCurrency = 'R$';var localeInfo = {"CountryCode":"BRA","CultureCode":"pt-BR","CurrencyLocale":{"RegionDisplayName":"Brazil","RegionName":"BR","RegionNativeName":"Brasil","TwoLetterIsoRegionName":"BR","CurrencyEnglishName":"Real","CurrencyNativeName"

In [6]:
pratos_beleaf = [i.text.strip() for i in beleaf_soup1.find_all('p', {'class' : "pratoNome"})]
print(pratos_beleaf)

['BURRITO BOWL', 'ASIA WOK', 'TUTU MINEIRO DE FEIJÃO PRETO', 'LASANHA BOLONHESA DO FUTURO™', 'NHOQUE DE BATATAS COM SHIITAKE E ESPINAFRE', 'FETTUCCINE DE ASPARGOS E TOMATE CEREJA', 'STROGONOFF LOW CARB', 'MOQUECA DE PUPUNHA', 'MINESTRONE', 'CREME DE CENOURA E MANDIOQUINHA', 'CREME DE ABÓBORA E CHIA', 'RAGU DE SHIITAKE', 'CURRY THAI DE LEGUMES E COGUMELOS', 'BURGUER DE SHIMEJI E BERINJELA', 'FEIJOADA', 'STROGONOFF DE COGUMELOS', 'CUSCUZ MARROQUINO', 'BOBÓ DE COGUMELOS NA MORANGA', 'FEIJÃO BRANCO ITALIANO', 'PAELLA NEGRA DE COGUMELOS', 'COMBO QUERIDINHOS(10 PRATOS)', 'COMBO PROTEÍNA(10 PRATOS)', 'COMBO MASSAS(12 PRATOS)', 'COMBO POUCA CALORIA(12 PRATOS)', 'BAIÃO DE DOIS', 'ESCALOPES DO FUTURO™', 'RISOTTO AO FUNGHI E ALHO NEGRO', 'PAD THAI', 'RISOTO DE GOJI BERRY E ARROZ NEGRO', 'BURGUER DE CENOURA', 'COGUMELO XADREZ', 'GRÃO DE BICO E MANGA AO MOLHO DE TAHINE', 'SHIMEJI REFOGADO']


In [7]:
preços_beleaf = [i.text.strip().strip("R").strip("$").replace(",", ".") for i in beleaf_soup1.find_all('p', {'class' : "pratoPrecoProduto"})]

print(preços_beleaf)

[' 19.90', ' 19.90', ' 19.90', ' 28.90', ' 26.30', ' 26.30', ' 22.90', ' 24.70', ' 14.90', ' 14.90', ' 14.90', ' 26.30', ' 25.70', ' 27.90', ' 23.90', ' 27.30', ' 22.90', ' 26.30', ' 21.90', ' 26.80']


In [8]:
acompanhamentos_beleaf = [i.text.strip().strip(",").strip(",") for i in beleaf_soup1.find_all('div', {'class' : "pratoSub Nome-complemento"})]

print(acompanhamentos_beleaf)


['Milho verde, tomate, cenoura e repolho roxo', 'Molho de laranja e arroz de jasmim', 'Molho acebolado, arroz cateto integral e couve refogada', 'Mussarela de castanhas', 'Molho de castanha de caju e mostarda dijon', 'Molho cremoso de castanhas e azeite trufado', 'Quinoa vermelha e legumes assados com capim-limão', 'Quinoa com espinafre e mix de legumes', 'Espinafre refogado', 'Castanha do pará e coentro', 'Sour cream de coco e limão', 'Polenta cremosa, vagem holandesa salteada', 'Arroz basmati, cebola roxa assada', 'Chutney de manga, aligot e legumes stir fry', 'Arroz com raspas de laranja, couve refogada', 'Arroz cateto integral com cenoura, brócolis e abobrinha refogados', 'Lentilha com legumes, couve refogada', 'Arroz cateto integral, espinafre refogado', 'Quinoa caprese, escarola refogada', 'Panache de legumes', '10 pratos mais pedidos', '5 pratos saudáveis e com proteína, com 2 unidades cada', 'Nossas 6 massas, com 2 unidades de cada', '9 pratos e 3 sopas com baixa caloria', 'Man

In [9]:
zippedList =  list(zip(pratos_beleaf, acompanhamentos_beleaf, preços_beleaf))

zippedList2 =  list(zip(pratos_beleaf))

In [10]:
dfbeleaf = pd.DataFrame(zippedList, columns = ['Prato', 'Acompanhameto' , 'Preço']) 

dfbeleaf2 = pd.DataFrame(zippedList2, columns = ['Prato']) 

In [11]:
dfbeleaf['Fonte']='Beleaf'

In [12]:
dfbeleaf

,Prato,Acompanhameto,Preço,Fonte
0,BURRITO BOWL,"Milho verde, tomate, cenoura e repolho roxo",19.90,Beleaf
1,ASIA WOK,Molho de laranja e arroz de jasmim,19.90,Beleaf
2,TUTU MINEIRO DE FEIJÃO PRETO,"Molho acebolado, arroz cateto integral e couve...",19.90,Beleaf
3,LASANHA BOLONHESA DO FUTURO™,Mussarela de castanhas,28.90,Beleaf
4,NHOQUE DE BATATAS COM SHIITAKE E ESPINAFRE,Molho de castanha de caju e mostarda dijon,26.30,Beleaf
5,FETTUCCINE DE ASPARGOS E TOMATE CEREJA,Molho cremoso de castanhas e azeite trufado,26.30,Beleaf
6,STROGONOFF LOW CARB,Quinoa vermelha e legumes assados com capim-limão,22.90,Beleaf
7,MOQUECA DE PUPUNHA,Quinoa com espinafre e mix de legumes,24.70,Beleaf
8,MINESTRONE,Espinafre refogado,14.90,Beleaf
9,CREME DE CENOURA E MANDIOQUINHA,Castanha do pará e coentro,14.90,Beleaf


In [13]:
dfbeleaf['Preço'] = dfbeleaf['Preço'].astype(float)



In [14]:

dfbeleaf.to_csv(r'C:\Users\marim\Documents\IRONHACK\MEGAHACKATHON\DFBELEAF1.csv', sep = ';', encoding = 'utf-8', index=False)



In [15]:
dfbeleaf2

,Prato
0,BURRITO BOWL
1,ASIA WOK
2,TUTU MINEIRO DE FEIJÃO PRETO
3,LASANHA BOLONHESA DO FUTURO™
4,NHOQUE DE BATATAS COM SHIITAKE E ESPINAFRE
5,FETTUCCINE DE ASPARGOS E TOMATE CEREJA
6,STROGONOFF LOW CARB
7,MOQUECA DE PUPUNHA
8,MINESTRONE
9,CREME DE CENOURA E MANDIOQUINHA


In [16]:
dfbeleaf2.to_csv(r'C:\Users\marim\Documents\IRONHACK\MEGAHACKATHON\DFBELEAF2.csv', sep=';', encoding= 'utf-8', index=False)

In [17]:
dfbeleaf['Preço'].mean()

23.124999999999996

In [32]:
pratos_ingredientes = []

stopwords = ["DE", "DO", "COM", "E", "AO", "NA", "COMBO", "QUERIDINHOS(10", "PROTEÍNA(10", "PRATOS)",
             "MASSAS(12", "POUCA", "CALORIA(12", "FUTURO™", 'REFOGADO']

for x in dfbeleaf2['Prato']:
    x = x.upper()
    x = x.split()  
    for value in x:
        if value not in stopwords:
            pratos_ingredientes.append(value) 

In [33]:
print(pratos_ingredientes)

['BURRITO', 'BOWL', 'ASIA', 'WOK', 'TUTU', 'MINEIRO', 'FEIJÃO', 'PRETO', 'LASANHA', 'BOLONHESA', 'NHOQUE', 'BATATAS', 'SHIITAKE', 'ESPINAFRE', 'FETTUCCINE', 'ASPARGOS', 'TOMATE', 'CEREJA', 'STROGONOFF', 'LOW', 'CARB', 'MOQUECA', 'PUPUNHA', 'MINESTRONE', 'CREME', 'CENOURA', 'MANDIOQUINHA', 'CREME', 'ABÓBORA', 'CHIA', 'RAGU', 'SHIITAKE', 'CURRY', 'THAI', 'LEGUMES', 'COGUMELOS', 'BURGUER', 'SHIMEJI', 'BERINJELA', 'FEIJOADA', 'STROGONOFF', 'COGUMELOS', 'CUSCUZ', 'MARROQUINO', 'BOBÓ', 'COGUMELOS', 'MORANGA', 'FEIJÃO', 'BRANCO', 'ITALIANO', 'PAELLA', 'NEGRA', 'COGUMELOS', 'BAIÃO', 'DOIS', 'ESCALOPES', 'RISOTTO', 'FUNGHI', 'ALHO', 'NEGRO', 'PAD', 'THAI', 'RISOTO', 'GOJI', 'BERRY', 'ARROZ', 'NEGRO', 'BURGUER', 'CENOURA', 'COGUMELO', 'XADREZ', 'GRÃO', 'BICO', 'MANGA', 'MOLHO', 'TAHINE', 'SHIMEJI']


In [34]:
def count_freq(list_str):
    token = [t for each_line in list_str for t in each_line.split(" ")]
    count_token = [token.count(i) for i in list(set(token))]
    term_freq_dict = dict(zip(set(token),count_token))
    return(term_freq_dict)

In [38]:
frequencia_tags = count_freq(pratos_ingredientes)

In [39]:
tagfreq = pd.DataFrame.from_dict(frequencia_tags, orient = "index", 
                                 columns = ["Frequência"]).sort_values("Frequência", ascending = False)

tagfreq

,Frequência
COGUMELOS,4
FEIJÃO,2
BURGUER,2
CREME,2
NEGRO,2
CENOURA,2
THAI,2
STROGONOFF,2
SHIMEJI,2
SHIITAKE,2


In [ ]:
stopwords = ["DE", "DO", "COM", "E", "EM", "AO", "NA", "COMBO", "QUERIDINHOS(10", "PROTEÍNA(10", "PRATOS)",
             "MASSAS(12", "POUCA", "CALORIA(12", "FUTURO™", "-", "–", "320G", "300G", "150G", 
           "250G", "350G", "200G", "320G", "150G", "UNID.", "1", "150GR", "INT.", "+", "DESFIADO", "DESFIADA",
            "KIT", "(CONSULTE", "DISPONIBILIDADE)", 
            "(04", "SALADAS)", "SEMANA", "PREMIUM", "-", "(10", "21/OUT", "REFEIÇÕES)", "(08", 
            "(SOMENTE", "PROTEÍNA)", "S/", "(02", "UNIDADES)", "FIT", "14/OUT", "A", "ALMOÇO", 
            "C/", "7", "À", "2", "-", "FALSO", "AOS", "1", "DILL", "3", 'REFOGADO']

In [60]:
megazord = pd.read_csv("C:/Users/marim/Documents/IRONHACK/MEGAHACKATHON/CAIOCHATO.csv", sep=";")

In [61]:
megazord.head()

,Prato,Preço,Calorias,Categoria
0,Hamburguer de fraldinha batata assada com lemo...,24.3,415.0,Carnes vermelhas
1,Almondega de carne com molho de tomate arroz d...,21.3,363.0,Carnes vermelhas
2,Kafta arroz de pupunha com legumes brocolis no...,23.7,351.0,Carnes vermelhas
3,File mignon suino com molho provencal risoto d...,22.3,507.0,Carnes vermelhas
4,Prato comida congelada hamburguer de fraldinha...,24.3,374.0,Carnes vermelhas


In [85]:
pratos_ingredientes_total = []

stopwords2 = ["DE", "DO", "COM", "E", "EM", "AO", "NA", "COMBO", "QUERIDINHOS(10", "PROTEÍNA(10", "PRATOS)",
             "MASSAS(12", "POUCA", "CALORIA(12", "FUTURO™", "-", "–", "320G", "300G", "150G", 
           "250G", "350G", "200G", "320G", "150G", "UNID.", "1", "150GR", "INT.", "+", "DESFIADO", "DESFIADA",
            "KIT", "(CONSULTE", "DISPONIBILIDADE)", 
            "(04", "SALADAS)", "SEMANA", "PREMIUM", "-", "(10", "21/OUT", "REFEIÇÕES)", "(08", 
            "(SOMENTE", "PROTEÍNA)", "S/", "(02", "UNIDADES)", "FIT", "14/OUT", "A", "ALMOÇO", 
            "C/", "7", "À", "2", "-", "FALSO", "AOS", "1", "DILL", "3", 'REFOGADO', "PRATO",
             "CONGELADA", "COMIDA", "MIX", "VAPOR", "NO"]

for x in megazord['Prato']:
    x = x.upper()
    x = x.split()  
    for value in x:
        if value not in stopwords2:
            pratos_ingredientes_total.append(value) 

In [86]:
print(pratos_ingredientes_total)

['HAMBURGUER', 'FRALDINHA', 'BATATA', 'ASSADA', 'LEMON', 'PEPPER', 'LEGUMES', 'BATATA', 'ASSADA', 'LEMON', 'PEPPER', 'LEGUMES', 'ALMONDEGA', 'CARNE', 'MOLHO', 'TOMATE', 'ARROZ', 'COUVE', 'FLOR', 'CENOURA', 'CASTANHA', 'CAJU', 'ABOBRINHA', 'TOMATE', 'ABOBRINHA', 'TOMATE', 'ARROZ', 'COUVE-FLOR', 'CENOURA', 'CASTANHA', 'CAJU', 'KAFTA', 'ARROZ', 'PUPUNHA', 'LEGUMES', 'BROCOLIS', 'BRÓCOLIS', 'ARROZ', 'PUPUNHA', 'LEGUMES', 'FILE', 'MIGNON', 'SUINO', 'MOLHO', 'PROVENCAL', 'RISOTO', 'LIMAO', 'SICILIANO', 'CENOURA', 'MOSTARDA', 'BALSAMICO', 'RISOTO', 'LIMÃO', 'SICILIANO', 'CENOURA', 'MOSTARDA', 'BALSÂMICO', 'HAMBURGUER', 'FRALDINHA', 'BATATA', 'ALECRIM', 'LEGUMES', 'BATATA', 'RÚSTICA', 'ALECRIM', 'LEGUMES', 'HAMBURGUER', 'FRALDINHA', 'BATATA', 'ALECRIM', 'BATATA', 'RÚSTICA', 'ALECRIM', 'HAMBURGUER', 'FRALDINHA', 'LEGUMES', 'ASSADOS', 'BROCOLIS', 'BRÓCOLIS', 'LEGUMES', 'ASSADOS', 'HAMBURGUER', 'FRALDINHA', 'ARROZ', 'INTEGRAL', 'AMENDOAS', 'BROCOLIS', 'LASCAS', 'ABOBORA', 'ARROZ', 'INTEGRAL', 'AM

In [87]:
def count_freq(list_str):
    token = [t for each_line in list_str for t in each_line.split(" ")]
    count_token = [token.count(i) for i in list(set(token))]
    term_freq_dict = dict(zip(set(token),count_token))
    return(term_freq_dict)

In [88]:
frequencia_tags2 = count_freq(pratos_ingredientes_total)

In [89]:
tagfreq2 = pd.DataFrame.from_dict(frequencia_tags2, orient = "index", 
                                  columns = ["Frequência"]).sort_values("Frequência", ascending = False)

tagfreq2

,Frequência
ARROZ,161
FRANGO,124
INTEGRAL,106
LEGUMES,96
BATATA,74
MOLHO,72
CARNE,66
PURÊ,51
BRÓCOLIS,48
MANDIOQUINHA,42


In [93]:
superfoods = ["BROCOLIS", "BRÓCOLIS", "SALMÃO", "GOJI", "QUINOA", "QUINUA", "CHIA", "COCO"
             "MANTEIGA", "CACAU"]


In [98]:
pratos_superfood = []


for y in megazord['Prato']:
    y = y.upper()
    y = y.split()  
    for value in y:
        if value in superfoods:
            pratos_superfood.append(value)

In [99]:
print(pratos_superfood)

['BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'QUINOA', 'QUINOA', 'BROCOLIS', 'BRÓCOLIS', 'QUINOA', 'QUINOA', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'QUINOA', 'QUINOA', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BROCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'SALMÃO', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'SALMÃO', 'BROCOLIS', 'QUINOA', 'BRÓCOLIS', 'BRÓCOLIS', 'SALMÃO', 'BRÓCOLIS', 'BRÓCOLIS', 'SALMÃO', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'BRÓCOLIS', 'QUINUA', 'BRÓCOLIS', 'BRÓCOLIS', 'QUINUA', 'QUINUA', 'CHIA', 'BRÓCOLIS', 'QUINUA', 'BRÓCOLIS', 'QUINUA', 'QUINUA', 'BRÓCOLIS', 'QUIN

In [100]:
frequencia_tags3 = count_freq(pratos_superfood)

In [101]:
tagfreq3 = pd.DataFrame.from_dict(frequencia_tags3, orient = "index", 
                                  columns = ["Frequência"]).sort_values("Frequência", ascending = False)

tagfreq3

,Frequência
BRÓCOLIS,48
BROCOLIS,22
QUINOA,10
QUINUA,10
SALMÃO,8
CHIA,2
